In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

In [2]:
sales_scaler = MinMaxScaler(feature_range=(0,1))
sales_scaler.fit(np.array([30,500]).reshape(-1,1))
loaded_model = load_model('./dynamic_pricing/')

# input in the form:  pd.series / list of dict 
def scale_sales(data): 
    return pd.Series(sales_scaler.transform(data.values.reshape(-1,1)).flatten(), index=data.index)

def inverse_scale_sales(data):
    return pd.Series(sales_scaler.inverse_transform(data.values.reshape(-1,1)).flatten(), index=data.index)

def scale_price(data, range=[]):
    if len(range) == 0:
        min = data.min()
        max = data.max()
    else: 
        min = range[0]
        max = range[1]
    scaler = MinMaxScaler(feature_range=(0,1))
    scaler.fit(np.array([min,max]).reshape(-1,1))
    return pd.Series(scaler.transform(data.values.reshape(-1,1)).flatten(), index=data.index)

def inverse_scale_price(data, range):
    min = range[0]
    max = range[1]
    scaler = MinMaxScaler(feature_range=(0,1))
    scaler.fit(np.array([min,max]).reshape(-1,1))
    return pd.Series(scaler.inverse_transform(data.values.reshape(-1,1)).flatten(), index=data.index)

In [9]:
data = pd.read_csv("scraped_dataset/dynamic_pricing.csv")

In [22]:
#input should be pandas Dataframe in the order: score, sales, price, Min Two rows
def predict_price(X, price_range):
    X['sales'] = scale_sales(X['sales'])
    X['price'] = scale_price(X['price'], price_range)
    actual_X = [[ X.iloc[-2].tolist(), X.iloc[-1].tolist() ]]
    result = loaded_model.predict(actual_X)
    actual_result = inverse_scale_price(pd.Series(result.flatten()), price_range)[0]
    return actual_result

In [23]:
X = data[data['id']==0][['score', 'sales', 'price']]
range = [X['price'].min(), X['price'].max()]
predict_price(X, range)

70657.15